In [1]:
from netCDF4 import Dataset
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import glob,os,pdb
import datetime as dt
%matplotlib ipympl
plt.close('all')

In [2]:
def calculate_bad_pixels(dark):
    dead = np.zeros(dark.shape[1:])
    hot = dead.copy()
    cold = dead.copy()
    high_std = dead.copy()
    temp_dark_std = np.nanstd(dark,0)
    temp_dark_mean = np.nanmean(dark,0)
    dead[temp_dark_std < 1] = 1  # Dead Pixels
    cold[
        np.logical_and(temp_dark_mean < 1000, temp_dark_std < 1)
    ] = 1  # COLD Pixels
    hot[
        np.logical_and(temp_dark_mean > 3000, temp_dark_std < 1)
    ] = 1  # HOT Pixels
    high_std[temp_dark_std > 5 * np.nanmean(temp_dark_std)] = (
        1
    )
    return {'dead':dead[:],'cold':cold[:],'hot':hot[:],'high_std':high_std[:]}
    

In [ ]:
os.chdir('/media/sata/methanesat/darks/')
ch4_files = sorted(list(Path('./2024').rglob('*CH4*.nc')))
ch4_files.extend(sorted(list(Path('./2025').rglob('*CH4*.nc'))))
o2_files = sorted(list(Path('./2024').rglob('*O2*.nc')))
o2_files.extend(sorted(list(Path('./2024').rglob('*O2*.nc'))))

ch4_files = np.array(ch4_files)
o2_files = np.array(o2_files)

ch4_bp = Dataset('../level1a_calibration_MSAT_20250722.0.0_CH4_BadPixelMap_CH4_20250722.nc','r')['BadPixelMap'][:] 
o2_bp = Dataset('../level1a_calibration_MSAT_20250722.0.0_O2_BadPixelMap_O2_20250722.nc','r')['BadPixelMap'][:] 


In [ ]:
plt.hist(ch4_bp.flatten(),bins=[0,1,3,5,11,24,50,100,150])

In [ ]:
plt.figure(); plt.pcolormesh(np.log(ch4_bp+0.000001)); plt.colorbar(); plt.show()

In [ ]:
o2_files

CH4:
CH4_00F20001
CH4_0A1E0001
CH4_063F0001
CH4_080F0001
CH4_05850001

O2:
O2_00F20001
O2_0A1E0001
O2_037E0001
O2_063F0001
O2_05850001

The following dark files have more bad pixels than usual and should be checked (and removed if necessary):
04A40001 07FA0001 05850001 091E0001 04900001 03BA0001 0A3E0001 05490001 
056F0001 06400001 03D10001 062B0001 0CA50001 05370001 06720001 0C750001 
06140001 04B00001 05220001 047C0001


# Make a list of the suspect files

In [ ]:
bad_pfx_list=sorted(list(set([
                    '0A1E','04A4','091E','037E','037C','034E','0135',
                    '0407','0135','04B0','03D1','0490','0A3E','056F',
                    '080F','0640','0537','063F','0585','03BA','00F2',
                    '07FA','0549','03D1','062B','0CA5','0537','0672',
                    '0C75','0614','04B0','0522','047C'
                    ])))
bad_o2_files = {}
bad_ch4_files = {}
for pfx in bad_pfx_list:
    bad_o2_files[pfx] = o2_files[np.where([str(s).split('/')[3] == f'{pfx}0001' for s in o2_files])[0].flatten()]
    bad_ch4_files[pfx] = ch4_files[np.where([str(s).split('/')[3] == f'{pfx}0001' for s in ch4_files])[0].flatten()]

In [ ]:
sorted(list(bad_o2_files.keys()))[15:]

# Plot all frames for each dark and save

In [ ]:
plt.close('all')
bad_o2_files = {}
bad_o2_file_list = open('noisy_o2_id','r').readlines()
for ifi,fi in enumerate(bad_o2_file_list):
    pfx = fi.split('/')[3]
    bad_o2_files[pfx] = fi.strip('\n')
for ip,pfx in enumerate(list(bad_o2_files.keys())):
    print(bad_o2_files[pfx])
    f = Dataset(bad_o2_files[pfx])
    dn = f['Frame/PixelData'][:]
    n_rows = max(((dn.shape[0])//4+1,1))
    fig,axs=plt.subplots(n_rows,4,figsize=(10,10))
    for i in range(dn.shape[0]):
        if dn.shape[0] < 4:
            ax = axs[i]
        else:
            ax = axs[i//4,i%4]
        g = ax.pcolormesh(dn[i],vmin=1000,vmax=1500); 
        plt.colorbar(g,ax=ax)
        ax.set_title(f'{i}')
    fig.tight_layout()
    fig.suptitle(f'{pfx} Dark Map')
    fname = '_'.join(bad_o2_files[pfx].split('/')[:-1])+bad_o2_files[pfx].split('/')[-1][:-3]+'_o2_darks.png'
    fig.savefig(f'figs/all_frames/{fname}')
    plt.close('all')
    del dn


2024/08/07/00C30001/po-2075/MethaneSAT_L0_O2_Dark_0_00C30001.nc
2024/09/01/00F20001/po-2075/MethaneSAT_L0_O2_Dark_0_00F20001.nc
2024/09/10/01350001/202409121318_po-2092/MethaneSAT_L0_O2_Dark_0_01350001.nc
2024/09/24/01A70001/202409291307_po-2117/MethaneSAT_L0_O2_Dark_0_01A70001.nc
2024/09/25/01B20001/202409281317_po-2116/MethaneSAT_L0_O2_Dark_0_01B20001.nc
2024/10/27/034E0001/202411022044_po-2197/MethaneSAT_L0_O2_Dark_0_034E0001.nc


In [ ]:
%debug

In [ ]:
plt.close('all')
bad_ch4_files = {}
bad_ch4_file_list = open('noisy_ch4_id','r').readlines()
for ifi,fi in enumerate(bad_ch4_file_list):
    pfx = fi.split('/')[3]
    bad_ch4_files[pfx] = fi.strip('\n')
for ip,pfx in enumerate(list(bad_ch4_files.keys())):
    print(pfx)
    f = Dataset(bad_ch4_files[pfx])
    dn = f['Frame/PixelData'][:]
    n_rows = max((dn.shape[0]//4+1,1))
    fig,axs=plt.subplots(n_rows,4,figsize=(10,10))
    for i in range(dn.shape[0]):
        if dn.shape[0] < 4:
            ax = axs[i]
        else:
            ax = axs[i//4,i%4]
        g = ax.pcolormesh(dn[i],vmin=1000,vmax=1500); 
        plt.colorbar(g,ax=ax)
        ax.set_title(f'{i}')
    fig.tight_layout()
    fig.suptitle(f'{pfx} Dark Map')
    fig.savefig(f'figs//all_frames/{bad_ch4_files[pfx][:-3]}__ch4_darks.png')
    plt.close('all')
    del dn

In [ ]:
os.makedirs(pfx,exist_ok=True)

# Plot the mean and standard deviation of the DN and the high STD pixels

In [ ]:
plt.close('all')
for pfx in list(bad_o2_files.keys()):
    print(pfx)
    f = Dataset(bad_o2_files[pfx][-1])
    dn = f['Frame/PixelData'][:]

    bp = calculate_bad_pixels(dn)
    high_std_bp = bp['high_std'][:]
    combined_bp = np.logical_or((high_std_bp > 0),(o2_bp > 0))

    fig,axs = plt.subplots(1,3,figsize=(8,2.7))
    g=axs[0].pcolormesh(dn.mean(0),vmin=1000,vmax=1500); plt.colorbar(g,ax=axs[0])
    g=axs[1].pcolormesh(dn.std(0),vmax=10); plt.colorbar(g,ax=axs[1])
    g=axs[2].pcolormesh(high_std_bp,vmin=0,vmax=1); plt.colorbar(g,ax=axs[2]);
    axs[0].set_title(f'Mean DN ({dn.shape[0]} Frm)')
    axs[1].set_title(f'Std DN')
    axs[2].set_title(f'STD BPM ({high_std_bp.sum()})')
    fig.suptitle(f'{pfx} 5*STD Bad Pixels')
    fig.tight_layout()
    fig.savefig(f'figs/{pfx}/{pfx}_mean_std_bpmask.png')
    plt.close('all')

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(8,3))
g = axs[0].pcolormesh(np.nanmean(dn[:-1],0),vmin=1000,vmax=1500); 
axs[0].set_title('first 19 frames')
axs[0].set_aspect('equal')
plt.colorbar(g,ax=axs[0])
g = axs[1].pcolormesh(dn[-1],vmin=1000,vmax=1500); 
axs[1].set_title('last frame')
axs[1].set_aspect('equal')
plt.colorbar(g,ax=axs[1])
fig.tight_layout()
fig.suptitle('01350001')

In [ ]:
plt.close('all')
for pfx in ['0043']:
    f = Dataset(o2_files[0])
    dn = f['Frame/PixelData'][:]

    bp = calculate_bad_pixels(dn)
    #bp = np.array(bp)
    high_std_bp = bp['high_std'][:]
    combined_bp = np.logical_or((high_std_bp > 0),(o2_bp > 0))

    fig,axs = plt.subplots(1,3,figsize=(8,2.7))
    g=axs[0].pcolormesh(dn.mean(0),vmin=1000,vmax=1500); plt.colorbar(g,ax=axs[0])
    g=axs[1].pcolormesh(dn.std(0),vmax=10); plt.colorbar(g,ax=axs[1])
    g=axs[2].pcolormesh(high_std_bp,vmin=0,vmax=1); plt.colorbar(g,ax=axs[2]);
    axs[0].set_title(f'Mean DN ({dn.shape[0]} Frm)')
    axs[1].set_title(f'Std DN')
    axs[2].set_title(f'STD BPM ({high_std_bp.sum()})')
    fig.suptitle(f'{pfx} 5*STD Bad Pixels')
    fig.tight_layout()

In [ ]:
f['Frame/PixelData'].dimensions

In [ ]:
plt.close('all')
mean_2 = Dataset('mean_darks/MethaneSAT_L0_O2_Dark_0_00430001_mean.nc','r')['mean_dark'][:]
std_2 = Dataset('mean_darks/MethaneSAT_L0_O2_Dark_0_00430001_mean.nc','r')['std_dark'][:]
plt.figure(); plt.pcolormesh(std_2-dn.std(0)),plt.colorbar(); plt.show()
plt.figure(); plt.pcolormesh(mean_2-dn.mean(0)); plt.colorbar(); plt.show()
plt.figure(); plt.hist((std_2-dn.std(0)).flatten(),bins=100); plt.show()
plt.figure(); plt.hist((mean_2-dn.mean(0)).flatten(),bins=100); plt.show()


In [ ]:
plt.close('all')
for pfx in ['037C']:
    f = Dataset(bad_ch4_files[pfx][-1])
    dn = f['Frame/PixelData'][:]

    bp = calculate_bad_pixels(dn)
    bp = np.array(bp)
    combined_bp = np.logical_or((bp > 0),(ch4_bp > 0))

    fig,axs = plt.subplots(1,3,figsize=(8,2.7))
    g=axs[0].pcolormesh(dn.mean(0),vmin=1000,vmax=1500); plt.colorbar(g,ax=axs[0])
    g=axs[1].pcolormesh(dn.std(0),vmax=10); plt.colorbar(g,ax=axs[1])
    g=axs[2].pcolormesh((bp>17)*(bp<33),vmin=0,vmax=1); plt.colorbar(g,ax=axs[2]);
    axs[0].set_title(f'Mean DN ({dn.shape[0]} Frm)')
    axs[1].set_title(f'Std DN')
    axs[2].set_title(f'STD BPM ({sum(sum((bp>17)*(bp<33)))})')
    fig.suptitle(f'{pfx} 5*STD Bad Pixels')
    fig.tight_layout()

In [ ]:
files = sorted(glob.glob('mean_darks/*_O2_*.nc'))

In [ ]:
fid = Dataset(files[0])
mean = fid['mean_dark'][:]
std = fid['std_dark'][:]
mean

In [ ]:
'_'.join(str(i) for i in range(10)[:-1])

In [ ]:
import os

In [ ]:
os.path.basename(files[0])